In [37]:
import pandas as pd
import sklearn.metrics as metrics
import math
import numpy as np
import pickle

In [38]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [39]:
def binary_metrics(pd_table, model, dataset):
    target = pd_table['target'].tolist()
    prediction = pd_table['prediction'].tolist()
    probability = pd_table['probability'].tolist()
    acc= metrics.accuracy_score(target,prediction)
    precision=metrics.precision_score(target, prediction)
    recall = metrics.recall_score(target, prediction)
    f1 = (2 * precision * recall) / (precision + recall)
    auc= metrics.roc_auc_score(target, probability)
    print(metrics.confusion_matrix(target, prediction))
    print(f'Accuracy = {np.round(acc, 4)}')
    print(f'Precision = {np.round(precision, 4)}')
    print(f'Recall = {np.round(recall, 4)}')
    print(f'F1 = {np.round(f1, 4)}')
    print(f'AUC = {np.round(auc, 4)}')
    
    result={
        'accuracy':acc,
        'precision':precision,
        'recall':recall,
        'f1':f1,
        'auc':auc,
        'model':model,
        'dataset':dataset,
        'target':target,
        'prediction':prediction,
        'probability':probability
    }
    filename = '{}_{}.pkl'.format(model,dataset)
    with open(filename, 'wb') as f:
        pickle.dump(result,f)

    return result

# STYLEGAN2 trained model

In [40]:
pd_xhululu_fake=pd.read_csv('out_xhululu_fake.csv')
pd_xhululu_real=pd.read_csv('out_xhululu_real.csv')
pd_xhululu_fake['target']= 0
pd_xhululu_real['target']= 1
pd_xhululu=pd.concat([pd_xhululu_fake,pd_xhululu_real],ignore_index = True)

pd_xhululu['probability'] = pd_xhululu['logit'].apply(lambda x : sigmoid(-x))

pd_xhululu['prediction'] = pd_xhululu['logit'].apply(lambda x : int(x<0))


In [41]:
pd_xhululu


,filename,logit,time,target,probability,prediction
0,../media-analysis-project-main/data/xhululu/re...,-4.359769,0.152972,0,0.987380,1
1,../media-analysis-project-main/data/xhululu/re...,3.423121,0.042867,0,0.031581,0
2,../media-analysis-project-main/data/xhululu/re...,3.100921,0.041247,0,0.043069,0
3,../media-analysis-project-main/data/xhululu/re...,4.414319,0.042667,0,0.011958,0
4,../media-analysis-project-main/data/xhululu/re...,-0.464620,0.041855,0,0.614110,1
...,...,...,...,...,...,...
19995,../media-analysis-project-main/data/xhululu/re...,-8.267615,0.059790,1,0.999743,1
19996,../media-analysis-project-main/data/xhululu/re...,-7.494914,0.058722,1,0.999444,1
19997,../media-analysis-project-main/data/xhululu/re...,-11.342032,0.059115,1,0.999988,1
19998,../media-analysis-project-main/data/xhululu/re...,-13.873199,0.059083,1,0.999999,1


In [42]:
target = pd_xhululu['target'].tolist()
prediction = pd_xhululu['prediction'].tolist()
probability = pd_xhululu['probability'].tolist()

In [43]:
result= binary_metrics(pd_xhululu, 'stylegan2', 'xhululu')


[[7314 2686]
 [  11 9989]]
Accuracy = 0.8652
Precision = 0.7881
Recall = 0.9989
F1 = 0.8811
AUC = 0.997


# PROGAN pretrined model

In [44]:
pd_xhululu_fake=pd.read_csv('out_xhululu_fake_progan.csv')
pd_xhululu_real=pd.read_csv('out_xhululu_real_progan.csv')
pd_xhululu_fake['target']= 0
pd_xhululu_real['target']= 1
pd_xhululu=pd.concat([pd_xhululu_fake,pd_xhululu_real],ignore_index = True)

pd_xhululu['prediction'] = pd_xhululu['logit'].apply(lambda x : int(x<0))
pd_xhululu['probability'] = pd_xhululu['logit'].apply(lambda x : sigmoid(-x))


In [45]:
result_progan = binary_metrics(pd_xhululu, 'resnet50_progan', 'xhululu')

[[1432 8568]
 [   7 9993]]
Accuracy = 0.5712
Precision = 0.5384
Recall = 0.9993
F1 = 0.6998
AUC = 0.9615


# whichfaceisreal StyleGAN2 pretrained

In [47]:
pd_fake=pd.read_csv('out_whichfaceisreal_fake.csv')
pd_real=pd.read_csv('out_whichfaceisreal_real.csv')
pd_fake['target']= 0
pd_real['target']= 1
pdtable=pd.concat([pd_fake,pd_real],ignore_index = True)

pdtable['prediction'] = pdtable['logit'].apply(lambda x : int(x<0))
pdtable['probability'] = pdtable['logit'].apply(lambda x : sigmoid(-x))

result = binary_metrics(pdtable, 'stylegan2', 'whichfaceisreal')

[[1000    0]
 [   0 1000]]
Accuracy = 1.0
Precision = 1.0
Recall = 1.0
F1 = 1.0
AUC = 1.0


# whichfaceisreal progan pretrained

In [48]:
pd_fake=pd.read_csv('out_whichfaceisreal_fake_progan.csv')
pd_real=pd.read_csv('out_whichfaceisreal_real_progan.csv')
pd_fake['target']= 0
pd_real['target']= 1
pdtable=pd.concat([pd_fake,pd_real],ignore_index = True)

pdtable['prediction'] = pdtable['logit'].apply(lambda x : int(x<0))
pdtable['probability'] = pdtable['logit'].apply(lambda x : sigmoid(-x))

result = binary_metrics(pdtable, 'progan', 'whichfaceisreal')

[[968  32]
 [  9 991]]
Accuracy = 0.9795
Precision = 0.9687
Recall = 0.991
F1 = 0.9797
AUC = 0.9989


# stargan StyleGAN2 pretrained

In [49]:
pd_fake=pd.read_csv('out_stargan_fake_stylegan2.csv')
pd_real=pd.read_csv('out_stargan_real_stylegan2.csv')
pd_fake['target']= 0
pd_real['target']= 1
pdtable=pd.concat([pd_fake,pd_real],ignore_index = True)

pdtable['prediction'] = pdtable['logit'].apply(lambda x : int(x<0))
pdtable['probability'] = pdtable['logit'].apply(lambda x : sigmoid(-x))

result = binary_metrics(pdtable, 'stylegan2', 'stargan')

[[1999    0]
 [   0 1999]]
Accuracy = 1.0
Precision = 1.0
Recall = 1.0
F1 = 1.0
AUC = 1.0


# whichfaceisreal ProGan pretrained

In [50]:
pd_fake=pd.read_csv('out_whichfaceisreal_fake_progan.csv')
pd_real=pd.read_csv('out_whichfaceisreal_real_progan.csv')
pd_fake['target']= 0
pd_real['target']= 1
pdtable=pd.concat([pd_fake,pd_real],ignore_index = True)

pdtable['prediction'] = pdtable['logit'].apply(lambda x : int(x<0))
pdtable['probability'] = pdtable['logit'].apply(lambda x : sigmoid(-x))

result = binary_metrics(pdtable, 'progan', 'whichfaceisreal')

[[968  32]
 [  9 991]]
Accuracy = 0.9795
Precision = 0.9687
Recall = 0.991
F1 = 0.9797
AUC = 0.9989


# deepfake StyleGan2 pretrained

In [51]:
pd_fake=pd.read_csv('out_deepfake_1_fake_stylegan2.csv')
pd_real=pd.read_csv('out_deepfake_0_real_stylegan2.csv')
pd_fake['target']= 0
pd_real['target']= 1
pdtable=pd.concat([pd_fake,pd_real],ignore_index = True)

pdtable['prediction'] = pdtable['logit'].apply(lambda x : int(x<0))
pdtable['probability'] = pdtable['logit'].apply(lambda x : sigmoid(-x))

result = binary_metrics(pdtable, 'stylegan2', 'deepfake')

[[   5 2693]
 [   0 2707]]
Accuracy = 0.5018
Precision = 0.5013
Recall = 1.0
F1 = 0.6678
AUC = 0.9325


# deepfake StyleGan2 pretrained

In [52]:
pd_fake=pd.read_csv('out_deepfake_1_fake_progan.csv')
pd_real=pd.read_csv('out_deepfake_0_real_progan.csv')
pd_fake['target']= 0
pd_real['target']= 1
pdtable=pd.concat([pd_fake,pd_real],ignore_index = True)

pdtable['prediction'] = pdtable['logit'].apply(lambda x : int(x<0))
pdtable['probability'] = pdtable['logit'].apply(lambda x : sigmoid(-x))

result = binary_metrics(pdtable, 'progan', 'deepfake')

[[  78 2620]
 [   0 2707]]
Accuracy = 0.5153
Precision = 0.5082
Recall = 1.0
F1 = 0.6739
AUC = 0.9362
